<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100"/>

<div>
    <font color=#690027 markdown="1">   
        <h1>SCHEIDINGSLIJN STOMATA OP BEZONDE EN BESCHADUWDE BLADEREN</h1> 
    </font>
</div>

<div class="alert alert-box alert-success">
In deze notebook zal je een rechte bepalen die de bezonde en beschaduwde bladeren (bij benadering) van elkaar scheidt. 
</div>

Krappa of crabwood is een snel groeiende boomsoort die veelvuldig voorkomt in het Amazonegebied. Volwassen exemplaren kunnen een diameter hebben van meer dan een meter en kunnen meer dan 40 meter hoog zijn. Het hout van hoge kwaliteit wordt gebruikt voor het maken van meubelen, vloeren, masten... Uit de schors wordt een koorstwerend middel gehaald. Uit de zaden produceert men een olie voor medicinale toepassingen, waaronder de behandeling van huidziekten en tetanos, en als afweermiddel voor insecten. 

<table><tr>
<td> <img src="images/andirobaamazonica.jpg" alt="Drawing" width="200"/></td>
<td> <img src="images/crabwoodtree.jpg" alt="Drawing" width="236"/> </td>
</tr></table>

<center>
Foto's: Mauroguanandi [Public domain] [2] en P. S. Sena [CC BY-SA 4.0] [3].
</center>

Omdat sommige klimaatmodellen een stijging van de temperatuur en een vermindering in regenval voorspellen in de komende decennia, is het belangrijk om te weten hoe deze bomen zich aanpassen aan veranderende omstandigheden. <br>
Wetenschappers Camargo en Marenco deden onderzoek in het Amazonewoud [1].<br>
Naast de invloed van seizoensgebonden regenval, bekeken ze ook stomatale kenmerken van bladeren onder bezonde en onder beschaduwde condities.<br> Hiervoor werden een aantal planten, opgekweekt in de schaduw, verplaatst naar vol zonlicht gedurende 60 dagen. Een andere groep planten werd in de schaduw gehouden. <br>De kenmerken van de stomata werden opgemeten op afdrukken van de bladeren gemaakt met transparante nagellak. 

### Nodige modules importeren

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression

from matplotlib import animation
from IPython.display import HTML

<div>
    <font color=#690027 markdown="1">   
        <h2>1. Inlezen van de data</h2> 
    </font>
</div>

Lees met de module `pandas` de dataset in.

In [ ]:
stomata = pd.read_csv("data/schaduwzon.csv", header="infer")  # in te lezen tabel heeft een hoofding

<div>
    <font color=#690027 markdown="1">   
        <h2>2. Stap voor stap op zoek naar de scheidingslijn</h2> 
    </font>
</div>

Net zoals de onderzoekers zet je de stomatale dichtheid uit tegenover de stomatale lengte. 

Net zoals in de vorige notebook wordt voor x$_{1}$ en x$_{2}$ met de gestandaardiseerde data gewerkt. 

In [ ]:
# data labelen
x1 = stomata["stomatale lengte"]          # kenmerk: lengte
x2 = stomata["stomatale dichtheid"]       # kenmerk: dichtheid
x1 = np.array(x1)          # kenmerk: lengte
x2 = np.array(x2)          # kenmerk: dichtheid

# standaardiseren
x1 = (x1 - np.mean(x1)) / np.std(x1)
x2 = (x2 - np.mean(x2)) / np.std(x2)

# labels 
y = stomata["milieu"]            # labels: tweede kolom van de oorspronkelijke tabel
y = np.array(y)
y = np.where(y == "zon", 1, 0)     # labels numeriek maken, zon:1, schaduw:0

In [ ]:
X = np.stack((x1, x2), axis = 1)    # juiste formaat
one_column = np.ones((X.shape[0],1))
X = np.concatenate((one_column, X), axis = 1)   # 1 toevoegen bij elk punt

In [ ]:
# trainingset met input X(x1, x2) en output y
print(X)
print(y)

<div>
    <font color=#690027 markdown="1">   
        <h3>2.1 Opbouw van het algoritme</h3> 
    </font>
</div>

Zo'n scheidingslijn wordt gezocht met een algoritme. Hier zie je hoe zo'n algoritme is opgebouwd. 

<div class="alert alert-box alert-info">
Het ML-systeem is een neuraal netwerk zonder verborgen laag en met activatiefunctie de sigmoid-functie.<br> Als foutenfunctie gebruikt me binary cross entropy.<br>
Om een rechte te vinden die de twee klassen van elkaar scheidt, vertrekt het ML-systeem van een willekeurig gekozen rechte. Dit gebeurt door de richtingscoëfficiënt en het snijpunt met de y-as van deze rechte willekeurig te kiezen.<br>  
    Het systeem wordt <em>getraind</em> met de trainingset (de inputs en de corresponderende labels): Voor elk punt van de trainingset wordt nagegaan hoeveel de fout bedraagt.  De coëfficiënten in de vergelijking van de rechte worden aangepast totdat de fout minimaal is. <br>
    De volledige trainingset wordt een aantal keer doorlopen. Zo'n keer noemt men een <em>epoch</em>. Het systeem <em>leert</em> gedurende deze <em>pogingen ('epochs')</em>.
</div>

Het neuraal netwerk maakt eerst een lineaire combinatie van de input met de weights.<br> Op dit resultaat werkt dan de **activatiefunctie** in. In dit neuraal netwerk is dat *sigmoid*. Voor elk datapunt geeft de sigmoid-functie een waarde terug tussen 0 en 1. Deze waarde geeft aan hoe zeker het systeem is dat het punt tot de klasse met label 1 behoort. 

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def predict(kenmerken, weights):
    """De voorspelling is een waarde die weergeeft hoe zeker het punt tot de klasse met label 1 behoort."""
    
    z = np.dot(kenmerken, weights.T) 
    voorspelling = sigmoid(z)
    
    return voorspelling 

Het systeem moet de fout kunnen berekenen na elke epoch. <br>Daartoe wordt voor elk punt het residu $y-\hat{y}$ berekend. Hierbij is $y$ de gegeven y-waarde en $\hat{y}$ de voorspelde waarde, nl. de waarde die men bekomt door de gegeven x-waarde in te vullen in de vergelijking van de rechte.<br> De kwadraten van de residu's worden bij elkaar opgeteld. Deze som gedeeld door het aantal datapunten is de gezochte fout. 

In [ ]:
def bc(kenmerken, labels, weights):
    """Fout binary crossentropy berekenen."""
    
    n = len(y)            # aantal punten
    predictions = predict(kenmerken, weights)            # huidige voorspelling
    
    #Take the error when label=1
    class1_cost = - labels * np.log(predictions)

    #Take the error when label=0
    class2_cost = (1 - labels) * np.log(1-predictions)
    
    #Take the sum of both costs
    kost = class1_cost + class2_cost

    #Take the average cost
    kost = kost.mean()

    return kost


# def loss(h, y):
#    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

In [ ]:
def gradient_descent(kenmerken, labels, weights, eta):
    """Aanpassing parameters q en m na voltooide epoch met learning rate eta."""
    
    n = len(labels)                                 # aantal punten is aantal waarden in lijst van labels y
    predictions = predict(kenmerken, weights)        # bekijk huidige predictions
     
    #2 Transpose kenmerken X from (50, 3) to (3, 50)
    # So we can multiply with the (50,1)  cost matrix.
    # Returns a (3,1) matrix holding 3 partial derivatives   nl. naar w1 en w2 en w0
    
    # berekenen van de partiële afgeleiden
    gradient = np.dot(kenmerken.T, (predictions - labels))
    gradient = gradient / n
    
    # waarden weights aanpassen
    weights = weights - eta *gradient
  
    # aangepaste weights teruggeven
    return weights

<div>
    <font color=#690027 markdown="1">   
        <h3>2.2 Uittesten van het algoritme van gradient descent voor meerdere epochs</h3> 
    </font>
</div>

Neem (0; 1; 0,2) als initiële waarde voor de *weights*. Voer gradient descent uit voor 200 epochs met learning rate 0,01 en waarbij de aanpassingen van de *weights* en de fout na elke *epoch* wordt getoond. 

In [ ]:
# algoritme testen
w = np.array([0, 1, 0.2])
eta = 0.01

for j in range(200):
    fout = bc(X,y,w)                       # binary crossentropy berekenen na elke epoch
    print(j, w, fout)                      # waarden weights en fout tonen na elke epoch
    w = gradient_descent(X, y, w, eta)     # waarden weights aanpassen na elke epoch 
    
print("De rechte snijdt de y-as in: %.3f" % (-w[0]/w[2]))
print("De rechte heeft als rico: %.3f" % (-w[1]/w[2]))
print("Binary crossentropy voor de rechte m.b.t. de data: %.4f" % fout)

In het voorbeeld zie je dat het aantal epochs mee zal bepalen hoe nauwkeurig de scheidingslijn wordt bepaald. De rechte die men heeft gevonden na bv. 20 epochs ligt nog zeer ver van de beoogde scheidingslijn. Kijk ook hoe de fout verloopt, zolang deze in absolute waarde blijft dalen is ze nog niet geminimaliseerd, het systeem *underfit* dan. Blijkbaar wordt de fout wel weer groter. Misschien is de *learning rate* te groot.

<div>
    <font color=#690027 markdown="1">   
        <h3>2.3 Hoe verandert de fout en de stand van de rechte gedurende het proces?</h3> 
    </font>
</div>

In [ ]:
def gradient_descent_proces(kenmerken, labels, weights, eta, epochs):
    """Proces doorlopen en gaandeweg ijsten maken van q, m en fout."""
    lijst_fout = [bc(kenmerken, labels, weights)]      # foutenlijst declareren en initialiseren
    lijst_weights = [weights]                          # lijst van weights declareren en initialiseren
    
    # Voor elke epoch lijsten aanvullen
    for i in range(epochs):
        weights = gradient_descent(kenmerken, labels, weights, eta)    # aangepaste parameters na epoch
        fout = bc(kenmerken, labels, weights)                      # kost na epoch 
        lijst_weights.append(weights)                           # aangepaste q toevoegen
        lijst_fout.append(fout)                           # deze kost toevoegen

    return [lijst_weights, lijst_fout]

Het proces doorlopen voor gekozen beginwaarden voor de gewichten, gekozen *learning rate* en gekozen aantal *epochs*.

In [ ]:
# initialisatie van de weights
w = np.array([0, 1, 0.2])

# vastleggen van aantal epochs en learning rate èta
eta = 0.01 
epochs = 1000

# algoritme lineaire regressie doorlopen voor keuze weights, èta en epochs
lijst_weights, lijst_fout = gradient_descent_proces(X, y, w, eta, epochs)

# scheidingslijn
print ("Doorgang y-as: %.3f" % (-lijst_weights[-1][0]/lijst_weights[-1][2]))
print ("Rico: %.3f" % (-lijst_weights[-1][1]/lijst_weights[-1][2]))    

# gemiddelde kwadratische afwijking regressielijn
print ("Geminimaliseerde fout: %.4f" %  lijst_fout[-1])

Een animatie:

In [ ]:
# alle rechten
xcoord =  np.linspace(-2, 2, 30) 

ycoord = []
for j in range(epochs):
    y_r = (-lijst_weights[j][1]/lijst_weights[j][2]) * xcoord + (-lijst_weights[j][0]/lijst_weights[j][2]) # y-waarde berekenen van alle x'n uit xcoord voor betreffende rechte
    ycoord.append(y_r)
ycoord = np.array(ycoord)    # type casting

# plot-venster initialiseren
fig, ax = plt.subplots()
line, = ax.plot(xcoord, ycoord[0], color="green")   # rechte plotten
ax.axis([x1.min()-1,x1.max()+1,x2.min()-1,x2.max()+1])  # bereik assen
plt.title("Amazone zon-schaduw gestandaardiseerd")
plt.xlabel("lengte stomata")              # xlabel geeft een omschrijving op de x-as
plt.ylabel("stomatale dichtheid")         # ylabel geeft een omschrijving op de y-as
plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25 (label 1)
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25 (label 0)

def animate(i):
    line.set_ydata(ycoord[i])    # update de vergelijking van de rechte  
    return line,

plt.close()  # om voorlopig plot-venster te sluiten, enkel animatiescherm nodig

anim = animation.FuncAnimation(fig, animate, repeat=False, frames=len(ycoord))
    
HTML(anim.to_jshtml())

In [ ]:
# grafiek evolutie fout
plt.figure(figsize=(10,8))
plt.plot(lijst_fout)
plt.xlabel('epoch')
plt.ylabel('binary cross entropy')
plt.title('Evolutie van de fout')
plt.show()

Experimenteer met de *learning rate* en het aantal *epochs*.

<div>
    <font color=#690027 markdown="1">   
        <h2>3. Classificatie met scikit-learn</h2> 
    </font>
</div>

In [ ]:
# data
x1 = stomata["stomatale lengte"]          # kenmerk: lengte
x2 = stomata["stomatale dichtheid"]       # kenmerk: dichtheid
x1 = np.array(x1)          # kenmerk: lengte
x2 = np.array(x2)          # kenmerk: dichtheid

# standaardiseren
x1 = (x1 - np.mean(x1)) / np.std(x1)
x2 = (x2 - np.mean(x2)) / np.std(x2)

y = stomata["milieu"]            # labels: tweede kolom van de oorspronkelijke tabel
y = np.array(y)
y = np.where(y == "zon", 1, 0)     # labels numeriek maken, zon:1, schaduw:0

In [ ]:
X = np.stack((x1, x2), axis = 1)    # juiste formaat

In [ ]:
# dataset met input X(x1, x2) en output y
print(X)
print(y)

In [ ]:
# Classificatie met de scikit-learn Logistic Regression Classifier
clf = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
clf.fit(X,y)

# bereik assen
x_min, x_max = x1.min() - .5, x1.max() + .5
y_min, y_max = x2.min() - .5, x2.max() + .5

h = .01  # stap 
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))     # rooster maken met xx en yy
# np.c_[xx.ravel(), yy.ravel() is alle mogelijke coördinaten gevormd met de xx'n en de yy's
z = clf.predict(np.c_[xx.ravel(), yy.ravel()])     # van al deze punten voorspellen tot welke klasse ze behoren

# resultaat plotten
# voorspellingen z staan in een rijmatrix, elke voorspelling moet weer overeenkomen met punt in rooster
z = z.reshape(xx.shape)    
plt.figure(figsize=(10, 8))
# plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# plot datapunten
# plt.scatter(x1, x2, c=y, edgecolors='k', cmap=plt.cm.Paired)
plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25
# plot scheidingslijn
# voorspelling z heeft voor elk punt in rooster waarde 0 of 1, rooster wordt zo in twee delen verdeeld
# grens tussen twee gebieden wordt getekend in zwart 
plt.contour(xx, yy, z, colors="black")            

plt.title("Carapa (gestandaardiseerd)")
plt.xlabel("stomatale lengte")
plt.ylabel("stomatale densiteit")
plt.legend(loc="lower left")

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

plt.show()

<div>
    <h2>Referentielijst</h2> 
</div>

[1] Camargo, Miguel Angelo Branco, & Marenco, Ricardo Antonio. (2012). Growth, leaf and stomatal traits of crabwood (Carapa guianensis Aubl.)<br> &nbsp; &nbsp; &nbsp; &nbsp; in central Amazonia. Revista Árvore, 36(1), 07-16. https://dx.doi.org/10.1590/S0100-67622012000100002 en via e-mail.<br>
[2] Mauroguanandi [Public domain]. https://commons.wikimedia.org/wiki/File:Andirobaamazonica.jpg. <br> &nbsp; &nbsp; &nbsp; &nbsp; Geraadpleegd op 13 augustus 2019 via Wikimedia Commons. <br>
[3] Sena, P. S..https://commons.wikimedia.org/wiki/File:Crabwood_tree.JPG. [CC BY-SA 4.0] Geraadpleegd op 13 augustus 2019 via Wikimedia Commons.

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>.

<div>
    <h2>Met steun van</h2> 
</div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>